In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Library

In [2]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=3509accf57c09435cf98e1392e42b0fd3d3545285ea9f52a7e7c37af0f14bc60
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=0ca77833f20ffca410cced2527d9eb8feb79856b68fcb6067025badeddaaf939
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models
import cv2
import torch.optim as optim
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import warnings
warnings.filterwarnings("ignore")
import fastai

from fvcore.nn import FlopCountAnalysis
from fastai.vision.all import *
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from typing import Tuple, Union
from collections import OrderedDict
from torch.cuda.amp import autocast
from glob import glob
from sklearn.model_selection import KFold

# Set up training

In [4]:
SEED = 85
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Split and Preprocess data

In [5]:
labels_train_val = pd.read_csv('/kaggle/input/data/train_val_list.txt')
labels_train_val.columns = ['Image_Index']
labels_test = pd.read_csv('/kaggle/input/data/test_list.txt')
labels_test.columns = ['Image_Index']
disease_labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']
# NIH Dataset Labels CSV File 
labels_df = pd.read_csv('/kaggle/input/data/Data_Entry_2017.csv')
labels_df.columns = ['Image_Index', 'Finding_Labels', 'Follow_Up_#', 'Patient_ID',
                  'Patient_Age', 'Patient_Gender', 'View_Position',
                  'Original_Image_Width', 'Original_Image_Height',
                  'Original_Image_Pixel_Spacing_X',
                  'Original_Image_Pixel_Spacing_Y', 'dfd']
# One hot encoding
for diseases in tqdm(disease_labels): 
    labels_df[diseases] = labels_df['Finding_Labels'].map(lambda result: 1 if diseases in result else 0)


100%|██████████| 14/14 [00:00<00:00, 28.89it/s]


In [6]:
labels_df['Finding_Labels'] = labels_df['Finding_Labels'].apply(lambda s: [l for l in str(s).split('|')])

num_glob = glob('/kaggle/input/data/*/images/*.png')
img_path = {os.path.basename(x): x for x in num_glob}

labels_df['Paths'] = labels_df['Image_Index'].map(img_path.get)
labels_df.head()

,Image_Index,Finding_Labels,Follow_Up_#,Patient_ID,Patient_Age,Patient_Gender,View_Position,Original_Image_Width,Original_Image_Height,Original_Image_Pixel_Spacing_X,...,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia,Paths
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143,...,0,0,0,0,0,1,0,0,0,/kaggle/input/data/images_001/images/00000001_000.png
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143,...,1,0,0,0,0,1,0,0,0,/kaggle/input/data/images_001/images/00000001_001.png
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168,...,0,0,1,0,0,1,0,0,0,/kaggle/input/data/images_001/images/00000001_002.png
3,00000002_000.png,[No Finding],0,2,81,M,PA,2500,2048,0.171,...,0,0,0,0,0,0,0,0,0,/kaggle/input/data/images_001/images/00000002_000.png
4,00000003_000.png,[Hernia],0,3,81,F,PA,2582,2991,0.143,...,0,0,0,0,0,0,0,0,1,/kaggle/input/data/images_001/images/00000003_000.png


In [7]:
unique_patients = np.unique(labels_df['Patient_ID'])
len(unique_patients)

30805

In [8]:
from sklearn.model_selection import train_test_split

# train-70
# val-10
# test-20
train_val_df_patients, test_df_patients = train_test_split(unique_patients, 
                                   test_size = 0.2,
                                   random_state = SEED,
                                    shuffle= True
                                   )
len(train_val_df_patients)

24644

In [9]:
train_val_df = labels_df[labels_df['Patient_ID'].isin(train_val_df_patients)]

In [10]:
labels_df.shape
print('train_val size', train_val_df.shape[0])
print('test size', labels_df.shape[0] - train_val_df.shape[0])

train_val size 89764
test size 22356


In [11]:
item_transforms = [
    Resize((224, 224)),
]

batch_transforms = [
    Flip(),
    Rotate(),
    Normalize.from_stats(*imagenet_stats),
]


def get_x(row):
    return row['Paths']

def get_y(row):
    labels = row[disease_labels].tolist()
    return labels

dblock = DataBlock(
    blocks=(ImageBlock, MultiCategoryBlock(encoded=True,vocab=disease_labels)),
                   splitter=RandomSplitter(valid_pct=0.125, seed=SEED),
                   get_x=get_x,
                   get_y=get_y,
                   item_tfms=item_transforms,
                   batch_tfms=batch_transforms
                  )
dls = dblock.dataloaders(train_val_df, bs=64)

# Our AWBs based VGG16_BN

In [12]:
class LKA(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dw_conv1 = nn.Conv2d(dim, dim, kernel_size=5, padding=2, groups=dim)
        self.dw_conv2 = nn.Conv2d(dim, dim, kernel_size=7, padding=9, dilation=3, groups=dim)
        self.pw_conv = nn.Conv2d(dim, dim, kernel_size=1)

    def forward(self, x):
        u = x
        x = self.dw_conv1(x)
        x = self.dw_conv2(x)
        x = self.pw_conv(x)
        return x * u  # Attention applied element-wise
class VGG16_LKA(nn.Module):
    def __init__(self, num_classes, dropout=None):
        super().__init__()
        base = models.vgg16_bn(pretrained=True)

        self.block1 = nn.Sequential(*base.features[0:6])    # 64
        self.block2 = nn.Sequential(*base.features[7:13])   # 128
        self.block3 = nn.Sequential(*base.features[14:23])  # 256
        self.block4 = nn.Sequential(*base.features[24:33])  # 512
        self.block5 = nn.Sequential(*base.features[34:43])  # 512

        # Insert LKA modules after block3, block4, block5
        self.lka1 = LKA(dim=64)
        self.lka2 = LKA(dim=128)
        self.lka3 = LKA(dim=256)
        self.lka4 = LKA(dim=512)
        self.lka5 = LKA(dim=512)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(dropout) if dropout else nn.Identity()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.lka1(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.block2(x)
        x = self.lka2(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.block3(x)
        x = self.lka3(x)  # Attention-enhanced features
        x = F.max_pool2d(x, 2, 2)

        x = self.block4(x)
        x = self.lka4(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.block5(x)
        x = self.lka5(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.pool(x).flatten(1)
        x = self.dropout(x)
        x = self.classifier(x)
        return x
model_vgg_lka = VGG16_LKA(num_classes=14, dropout=0.5)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:02<00:00, 211MB/s] 


# Focal Loss

In [13]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        pt = torch.exp(-bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Train stage 2 with FocalLoss

In [14]:
learn = Learner(
    dls,
    model_vgg_lka,
    loss_func = FocalLoss(alpha=1.0, gamma=2.0),
    metrics=[accuracy_multi, F1ScoreMulti(), RocAucMulti()],
    cbs=[
        SaveModelCallback(monitor='valid_loss', fname='best_model', min_delta=0.0001, with_opt=False),
        EarlyStoppingCallback(monitor='valid_loss', min_delta=0.001, patience=5),
        ShowGraphCallback()
    ]
)
learn = learn.load('/kaggle/input/vgg-lka-5-block-stage-1/best_model_vgg_lka_5_block_stage_1', weights_only=False)

In [15]:
# Move model to GPU
model = learn.model.eval().cuda()  # Unwrap DataParallel and move to GPU

# Create a dummy input on the same device
sample_input = torch.randn(1, 3, 224, 224).cuda()

# Compute FLOPs
flops = FlopCountAnalysis(model, sample_input)
print(f"Total FLOPs: {flops.total() / 1e9:.2f} GFLOPs")  # Display in GFLOPs

Total FLOPs: 16.70 GFLOPs


In [ ]:
learn.unfreeze()
learn.model
sum(p.numel() for p in learn.model.parameters() if p.requires_grad)

In [ ]:
learn.fit_one_cycle(10, slice(2e-5, 8e-5))